<a href="https://colab.research.google.com/github/hyuna0926/cp2_phase2/blob/main/tunning/tunning_implicit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필요 라이브러리 불러오기

In [1]:
! pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 35.3 MB/s eta 0:00:00


In [80]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from scipy import sparse
from scipy.sparse import csr_matrix
from tqdm.notebook import tqdm
import numpy as np

from implicit.als import AlternatingLeastSquares as ALS
import implicit
import random
import os

# 2. 데이터 불러오기 및 분리하기


In [ ]:
# df1 = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/model_data.parquet') #구매한것만

In [81]:
# add_to_cart까지 포함한것
df = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/cart_purchase.parquet')

In [ ]:
df.head()

,created_at,customer_id,product_id,p_idx,event,c_idx
0,2018-07-29 15:22:01,5868,54728,29317,purchase,3039
1,2021-01-15 19:52:40,82831,54728,29317,purchase,41968
2,2016-10-20 10:16:35,82831,34982,8394,purchase,41968
3,2018-08-26 14:09:52,82831,8627,9052,purchase,41968
4,2018-03-17 15:08:08,82831,46703,23994,purchase,41968


In [ ]:
df.shape

(1876781, 6)

In [ ]:
df['customer_id'].nunique()

50698

In [ ]:
df['p_idx'].nunique()

31121

- 구매했으니 1로 둠

In [82]:
df['values']=1

In [ ]:
df.head()

,created_at,customer_id,product_id,p_idx,event,c_idx,values
0,2018-07-29 15:22:01,5868,54728,29317,purchase,3039,1
1,2021-01-15 19:52:40,82831,54728,29317,purchase,41968,1
2,2016-10-20 10:16:35,82831,34982,8394,purchase,41968,1
3,2018-08-26 14:09:52,82831,8627,9052,purchase,41968,1
4,2018-03-17 15:08:08,82831,46703,23994,purchase,41968,1


- groupby로 만들어서 train, test customer_id 비율을 맞춰줬음
  - 비율이 안맞아서 검증셋에 없는 customer_id가 있지만 그래도 균등하게 나눠진듯

In [83]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [84]:
df_group=df.groupby(['c_idx'], as_index = False).count()
upper_20 = list(df_group.query("values >= 20").c_idx) #22403
lower_20 = list(df_group.query("values < 20").c_idx) #28295

In [85]:
df_group=df.groupby(['c_idx'], as_index = False).count()

upper = df[df['c_idx'].isin(df_group.query("values >= 20").c_idx)]


test = upper.groupby('c_idx').sample(frac=0.2,random_state=42)
train = upper.drop(test.index)

user_20 = sparse.csr_matrix((train['values'],(train['c_idx'], train['p_idx'])))
test_user_20 = sparse.csr_matrix((test['values'],(test['c_idx'], test['p_idx'])))

In [86]:
# 검증셋 만들기
test_df = test.groupby('c_idx')['p_idx'].unique().to_frame().reset_index()
test_df

,c_idx,p_idx
0,0,"[3393, 14555, 9702, 25009, 24439, 19617, 8865,..."
1,4,"[13807, 1213, 3051, 23329]"
2,6,"[20858, 27397, 18336, 5254, 30369, 3425, 889, ..."
3,7,"[28436, 438, 8041, 16482, 17893, 27284, 10339,..."
4,9,"[29679, 998, 7848, 26990, 25655, 22110, 11228]"
...,...,...
22398,50689,"[7186, 12438, 23458, 19032, 25009, 11686, 5743..."
22399,50690,"[3853, 14872, 29576, 14140, 243, 2125, 21300, ..."
22400,50693,"[22738, 886, 22855, 23942, 2454, 25744]"
22401,50694,"[8074, 18765, 571, 21249, 784, 1870, 16300, 83..."


# 3. implicit 라이브러리 튜닝함수

In [87]:
def tunning_pre_hit(user_items,params,n_iters=10):
  results =[]
  for _ in tqdm(range(n_iters)):
    factor = np.random.choice(params['factor'])
    alpha = np.random.choice(params['alpha'])
    regularization = np.random.choice(params['regularization'])
    iteration = np.random.choice(params['iteration'])

    ALS = implicit.als.AlternatingLeastSquares(factors=factor, alpha=alpha, regularization=regularization, iterations=iteration,
              random_state=42, calculate_training_loss=True)
    ALS.fit(user_items)

    #hit_rate, precision
    total= 0
    user = 0
    precision=0
    for c_idx in range(500):
      count=0 # for문 돌 때마다 리셋
      if c_idx in upper_20:
        recommend = ALS.recommend(c_idx, user_20[c_idx],N=25)[0]
        buy_test = test_df[test_df['c_idx']==c_idx].p_idx.values[0]
        user+=1
        for i in buy_test:
          for j in recommend:
            if i==j:
              count+=1
              precision+=1 #precision
      if count >=1: #count가 1 이상이면 hit한 것이니까 전체에 1 추가
        total+=1
      
    hit_rate = total/user
    precision_mean = precision/user

    
    result = [factor, alpha, regularization, iteration, hit_rate, precision_mean]
    results.append(result)
  frame = pd.DataFrame(results, columns=['factor','alpha','regularization','iteration','hit_rate','precision'])
  frame = frame.sort_values('precision', ascending=False)[:5]

  return frame

In [88]:
params = {
    'factor': np.arange(100,210,10),
    'alpha' : np.arange(1,41,5),
    'regularization' : np.arange(1,41,5),
    'iteration' : np.arange(10,50,5)
}

In [89]:
tunning_pre_hit(user_20,params,n_iters=5)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

,factor,alpha,regularization,iteration,hit_rate,precision
2,200,1,31,35,0.224576,0.368644
4,100,26,31,20,0.173729,0.216102
3,190,36,21,30,0.161017,0.203390
1,140,16,11,20,0.144068,0.156780
0,200,26,36,40,0.127119,0.152542
